# Trabajando con instancia de base de datos postgres en el servicio RDS de AWS

**Instalamos el modulo boto3 para la conexion y configuracion de credenciales de AWS**

In [ ]:
!pip install boto3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 132 kB 5.2 MB/s 
     |████████████████████████████████| 79 kB 8.3 MB/s 
     |████████████████████████████████| 9.2 MB 49.1 MB/s 
     |████████████████████████████████| 140 kB 65.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.


**Configuracion de las credenciales**

In [ ]:
import boto3
from aws_credentials import ACCESS_KEY_ID, SECRET_ACCESS_KEY, REGION_NAME

client = boto3.client(
    'rds',
    aws_access_key_id=ACCESS_KEY_ID,
    aws_secret_access_key=SECRET_ACCESS_KEY,
    region_name = REGION_NAME
)

**Conexion con la base de datos**

In [ ]:
import psycopg2
from db_info import ENDPOINT, PORT, DB_USER, DB_PASSWORD, DBNAME

try:
    # Se crea el objeto de conexion
    conn = psycopg2.connect(host=ENDPOINT, port=PORT, user=DB_USER, password=DB_PASSWORD)
    conn.autocommit=True
    # Se crea el objeto cursor
    cur = conn.cursor()
except Exception as e:
    print("Database connection failed due to {}".format(e))

**Creamos una base de datos llamada awsmoises**

In [ ]:
cur.execute("""CREATE DATABASE awsmoises""")

Database connection failed due to database "awsmoises" already exists



In [ ]:
conn.close()

**Ahora tenemos que acceder a la base de datos especifica**

In [ ]:
try:
    conn = psycopg2.connect(host=ENDPOINT, port=PORT, user=DB_USER, password=DB_PASSWORD, database=DBNAME)
    conn.autocommit=True
    cur = conn.cursor()
except Exception as e:
    print("Database connection failed due to {}".format(e))

**Ahora vamos a crear una tabla en la base de datos llamada country**

In [ ]:
cur.execute("""CREATE TABLE country(
    id int PRIMARY KEY NOT NULL,
    ccode char(2),
    name varchar
) """)

**Ahora vamos a eliminar esa tabla**

In [ ]:
cur.execute("""DROP TABLE instructor """)

**1) Crear la tabla INSTRUCTOR como se ha definido anteriormente. Haz que ins_id sea la clave primaria, y asegúrate de que lastname ("apellido") y firstname ("nombre") sean not null.**

In [ ]:
conn = psycopg2.connect(host=ENDPOINT, port=PORT, user=DB_USER, password=DB_PASSWORD, database=DBNAME)
conn.autocommit=True
cur = conn.cursor()

cur.execute("""CREATE TABLE instructor(
    ins_id int PRIMARY KEY NOT NULL,
    lastname varchar NOT NULL,
    firstname varchar NOT NULL,
    city varchar,
    country char(2)
) """)

**2) Inserta tres filas a la vez en la tabla INSTRUCTOR**

In [ ]:
cur.execute("""INSERT INTO instructor
    (ins_id, lastname, firstname, city, country)
VALUES
    (1, 'Dos santos', 'Neymar', 'Santos', 'BR'),
    (2, 'Gonzales', 'Tyrone', 'Maracay', 'VE'),
    (3, 'Arango', 'Juan', 'Maracay', 'VE')
""")

**Selecciona todas las filas de la tabla INSTRUCTOR**

In [ ]:
cur.execute("""SELECT * FROM instructor""")
query_results = cur.fetchall()

for row in query_results:
    print(row)

(1, 'Dos santos', 'Neymar', 'Santos', 'BR')
(2, 'Gonzales', 'Tyrone', 'Maracay', 'VE')
(3, 'Arango', 'Juan', 'Maracay', 'VE')


**Selecciona firstname, lastaname y country donde city sea Maracay**

In [ ]:
cur.execute("""SELECT firstname, lastname, country FROM instructor
WHERE city='Maracay' """)

query_results = cur.fetchall()

for row in query_results:
    print(row)

('Tyrone', 'Gonzales', 'VE')
('Juan', 'Arango', 'VE')


**Actualiza la fila de Neymar y cambia su ciudad (“city”) a Barcelona.**

In [ ]:
cur.execute(""" UPDATE instructor
                SET city='Barcelona',
                    country='ES'
                WHERE ins_id=1 """)

**Agregando mas datos**

In [ ]:
cur.execute(""" INSERT INTO instructor
                    (ins_id, lastname, firstname, city, country)
                VALUES
                    (4, 'Messi', 'Lionel', 'Rosario', 'AR'),
                    (5, 'Nazario', 'Ronaldo', 'Rio de Janeiro', 'BR'),
                    (6, 'Zinedine', 'Zidane', 'Paris', 'FR'),
                    (7, 'Cristiano', 'Ronaldo', 'Porto', 'PG') """)

In [ ]:
cur.execute(""" SELECT * FROM instructor
                WHERE firstname='Ronaldo' """)

fname = cur.fetchall()

for rec in fname:
    print(rec)

(5, 'Nazario', 'Ronaldo', 'Rio de Janeiro', 'BR')
(7, 'Cristiano', 'Ronaldo', 'Porto', 'PG')


**Elimina la fila de Cristiano y Zidane de la tabla.**

In [ ]:
cur.execute(""" DELETE FROM instructor
                WHERE ins_id IN (6, 7) """)

## Trabajando con almacenamiento de archivos en servicio S3 de AWS

**Creamos el cliente para acceder al servicio S3 con python**

In [ ]:
import boto3
from aws_credentials import ACCESS_KEY_ID, SECRET_ACCESS_KEY, REGION_NAME

s3 = boto3.client(
    's3',
    aws_access_key_id=ACCESS_KEY_ID,
    aws_secret_access_key=SECRET_ACCESS_KEY,
    region_name=REGION_NAME
)

**Listamos los buckets o instancias de almacenamiento en S3**

In [ ]:
# Retrieve the list of existing buckets
#s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  mi-carpeta-awss3


**Cargamos un archivo desde nuestro local a el bucket que queremos**

In [ ]:
s3.upload_file(
    Filename="watch.got.csv",
    Bucket="mi-carpeta-awss3",
    Key="watch.got.csv"
)

**Descargamos un archivo del bucket de S3**

In [ ]:
s3.download_file(
    Filename="watch.got.csv",
    Bucket="mi-carpeta-awss3",
    Key="watch.got.csv"
)

**Creamos la tabla donde vamos a importar los datos del archivo .csv con las mismas columnas y mismo tipo de dato**

In [ ]:
cur.execute(""" CREATE TABLE csv_file(
                    id int PRIMARY KEY NOT NULL,
                    name text,
                    lastname text,
                    wgot boolean
)""")

**Abrimos el archivo con open**

In [ ]:
watch_dog_csv = open("watch.got.csv", "r")

print(watch_dog_csv.read())

id,name,lastname,wgot
1,"Moises","Sira",True
2,"Jorge","Sira",False
3,"Brityi","Vasquz",True
4,"Thais","Rosales",False
5,"Alexandra","Rosales",False
6,"Alexander","Sira",False



In [ ]:
# Esta opcion se usa cuando se trabaja con archivos locales y base de datos local
cur.copy_from(watch_dog_csv, "csv_file", sep=",", columns=('id', 'name', 'lastname', 'wgot'))

TypeError: ignored

**Instalamos la extension aws_s3 para poder extraer los datos del bucket del repositorio S3 de amazon**

*Para ver las extenciones instaladas en psql ejecutamos el comando \dx*

In [ ]:
cur.execute(""" CREATE EXTENSION aws_s3 CASCADE """)

**Para importar la data del archivo csv a la base de datos usamos la funcion aws_s3.table_import_from_s3**

*Documentacion:*

[Importación de datos de Amazon S3 en una instancia de base de datos de RDS para PostgreSQL](https://docs.aws.amazon.com/es_es/AmazonRDS/latest/UserGuide/USER_PostgreSQL.S3Import.html#USER_PostgreSQL.S3Import.FileFormats)

[Easily load data from an S3 bucket into Postgres using the aws_s3 extension](https://medium.com/analytics-vidhya/easily-load-data-from-an-s3-bucket-into-postgres-using-the-aws-s3-extension-17610c660790)

In [ ]:
cur.execute(f""" SELECT aws_s3.table_import_from_s3(
                    'csv_file',
                    'id,name,lastname,wgot', 
                    '(format csv, header true)',
                    'mi-carpeta-awss3',
                    'watch.got.csv',
                    'sa-east-1',
                    '{ACCESS_KEY_ID}',
                    '{SECRET_ACCESS_KEY}'
                ) """)